The Router Chain in [LangChain](https://www.langchain.com/) is used with [Anyscale Endpoints](https://console.endpoints.anyscale.com/) 
to write a Mixed-expert Q & A AI system. Each "subject" such as Math, Physics, Astronomy, Cosmology, Football, or Computer Science has a specific LangChain bound to specific Llama 2 type of model. 

Anyscale Endpoints support these open-source LLM models today:
 * meta-llama/Llama-2-7b-chat-hf
 * meta-llama/Llama-2-13b-chat-hf
 * meta-llama/Llama-2-70b-chat-hf
 * codellama/CodeLlama-34b-Instruct-hf
 
<img src="anyscale_endpoints.png" height="35%" width="%75">
 
You can think of these specific subject matter experts as LLMs that have been fine-tuned models with specific tasks or suitable to only answers questions related to subject area expertise.

<img src="router_chain.png">

This particular example is an extension of Router Chain disscussed 
in the [Deeplearning.ai and LangChain course](https://learn.deeplearning.ai/langchain/lesson/4/chains) by Andrew Ng and Harrison Chase, modified and extended here to work with [Anyscale Endpoints](https://console.endpoints.anyscale.com/)

In [1]:
import warnings
import os

import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv, find_dotenv
from router_prompts import (PromptInfo, MULTI_PROMPT_ROUTER_TEMPLATE)

import gradio as gr

In [2]:
_ = load_dotenv(find_dotenv()) # read local .env file
MODEL = 'meta-llama/Llama-2-13b-chat-hf'
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))

In [3]:
# Create your default model
llm_default = ChatOpenAI(temperature=0.9, model_name=MODEL, streaming=True)

Build destination chains, with each unique LLMChain
bound to a specify type of Llama 2 model and
the subject matter expert prompt to answer questions. 
For example, a physics question will be associated with a physics
LLMChain and related trained model.

In [4]:
# dictionary of chains
destination_chains = {}
prompt_infos = PromptInfo().prompt_infos
for p_info in prompt_infos:
     # name of the chain for the subject
    name = p_info["name"] 
    
    # subject template
    prompt_template = p_info["prompt_template"] 
    
    # fine-tuned subject-expert model
    model_t = p_info["model"] 
    llm_t = ChatOpenAI(temperature=0.9, model_name=model_t, streaming=True)
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm_t, prompt=prompt) # create the chain and its bound model
    destination_chains[name] = chain  

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [5]:
# default prompt and chain if none matches
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm_default, prompt=default_prompt)

In [6]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm_default, router_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [7]:
questions = [
             "what's angular momentum",
             "Explain step by step how to compute a prime number",
             "Write a Python program to quicksort a large array of numbers",
             "When was Armstice treaty signed and where and why",
             "What can say about them Arsenal Football Club"
            ]

In [ ]:
def send_prompt(text: str):
    return chain.run(text)
    

In [ ]:
mutli_demo= gr.Interface(fn=send_prompt, inputs="text", outputs="text")
mutli_demo.launch()   

### Use this for debugging 

In [8]:
for question in questions:
    print("---" * 5)
    print(f"question: {question}")
    print(chain.run(question))

---------------
question: what's angular momentum


> Entering new MultiPromptChain chain...
physics: {'input': "what's angular momentum"}
> Finished chain.
Hello! I'm glad you asked me about angular momentum! Angular momentum is a fundamental concept in physics that describes the tendency of an object to continue rotating or revolving around a central point. It is a measure of the amount of rotation an object has, and it is determined by the object's mass, radius, and rate of rotation.
The formula for angular momentum is:
L = r x p

Where L is the angular momentum, r is the distance from the center of rotation to the point where the momentum is measured, and p is the momentum of the object at that point.

Angular momentum is important in many areas of physics, including mechanics, electromagnetism, and quantum mechanics. It plays a key role in the behavior of objects in circular motion, such as planets orbiting around a star or a car making a turn on a road.
I hope that helps! Let me 